# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

/usr/lib/python3/dist-packages/matplotlib/__init__.py:874: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [4]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada una de las combinaciones de título y estado de título, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [5]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
347337,PmJVYER,Otro,Graduado,1.0
503807,1QdGq2B,Secundario,Graduado,2.0
496883,wVkAbYl,Terciario/Técnico,En Curso,3.0
292178,ekOKAN9,Secundario,En Curso,2.0
348697,E53zLo,Terciario/Técnico,En Curso,3.0


Analizamos la cantidad de IDs duplicados

In [6]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [7]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [8]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
59542,X9rzxQ8,Secundario,Abandonado,2.0
322878,2zPEGPQ,Terciario/Técnico,Graduado,3.0
296080,8jKV8M,Terciario/Técnico,Graduado,3.0
171975,vVjb3NX,Secundario,Graduado,2.0
33873,6wk4kv,Terciario/Técnico,Graduado,3.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [9]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [10]:
postulantes_educ['estado'].value_counts()

Graduado      253166
En Curso      148362
Abandonado     46381
Name: estado, dtype: int64

In [11]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios,esta_estudiando
41954,RzJoXYR,Secundario,Graduado,2.0,0.0
55535,N0Q8KL,Universitario,En Curso,4.0,1.0
76546,Ezl8a6b,Universitario,En Curso,4.0,1.0
657682,1Qk3PAj,Secundario,Graduado,2.0,0.0
160807,QNMkZVe,Universitario,Abandonado,4.0,0.0


Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'orden_estudio' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,estudios,esta_estudiando
491183,eVvja9,3.0,0.0
78683,2zkEW3o,2.0,0.0
560077,ak4MqLN,2.0,0.0
48198,ZD869lY,4.0,1.0
262858,4rdZmL5,4.0,0.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
335988,6rQ3VBx,1984-06-04,FEM
163257,2zQbONw,1988-08-25,MASC
483831,Bm5AbME,1990-08-27,FEM
37989,85Wjq3,1990-06-06,MASC
388532,ZDrmzL5,1994-06-23,FEM


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
453203,JBekZAb,39,MASC
157144,A3RoAZ1,25,FEM
109019,EzZkPNz,27,MASC
380749,dYjljOP,25,MASC
291484,ekO2Oz9,26,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
215070,0zPqX5Y,22,MASC,1.0
250199,JBr1kEb,22,FEM,1.0
392268,JBeKwBk,32,MASC,3.0
50694,8KPOVM,42,FEM,4.0
198408,VNxRjxl,20,FEM,1.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
77591,QNawQmN,MASC,2.0
356947,1krwvz,MASC,3.0
470221,YjVZ991,FEM,1.0
223154,rmdGGKz,MASC,1.0
383739,Oqr8VXr,FEM,1.0


In [27]:
#asigno a variable numerica el sexo

In [25]:
postulantes_gen_nac["sexo"].value_counts()

FEM           250194
MASC          226713
NO_DECLARA      1094
Name: sexo, dtype: int64

Pasamos a valor numérico el sexo de los postulantes.

- FEM = 1
- MASC = 2
- NO_DECLARA = 0

In [26]:
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'FEM'),'sexo'] = 1

postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'MASC'),'sexo'] = 2

postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'NO_DECLARA'), 'sexo'] = 0

postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
379565,akjpODZ,1,1.0
467533,BmBL66R,2,1.0
180098,QNEjqqz,2,3.0
210724,rmdRJNb,1,4.0
150140,wVj3xGZ,2,1.0


In [27]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null int64
rango_edad      478001 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


Liberamos memoria para mejorar la performance

In [28]:
postulantes_educ_2=0
postulantes_gen_nac_2=0
fecha_nac=0
año_nac=0
edad_min=0
edad_max=0
condicion_final=0
rango_18_25=0
rango_26_30=0
rango_31_40=0
rango_41_68=0

### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [29]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,a0XaWD,7.0,0.0,2,4.0
1,4reGo5z,7.0,0.0,1,2.0
2,X9lpKkb,7.0,0.0,2,4.0
3,RzMXJ4E,7.0,0.0,1,4.0
4,akOA6b9,7.0,0.0,2,4.0


In [30]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438451 entries, 0 to 438450
Data columns (total 5 columns):
idpostulante       438451 non-null object
estudios           438451 non-null float64
esta_estudiando    438451 non-null float64
sexo               438451 non-null int64
rango_edad         438451 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 20.1+ MB


### DF 4: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [31]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [32]:
len(postulaciones)

6604534

In [33]:
len(postulaciones.drop_duplicates(['idaviso','idpostulante']))

6603752

In [34]:
#como hay solo una diferencia de 800(aprox) duplicados, los tiro
postulaciones.drop_duplicates(['idaviso','idpostulante'], inplace=True)
len(postulaciones)

6603752

In [35]:
#elimino la columna fechapostulacion ya que no nos brinda nada.
postulaciones.drop(columns={'fechapostulacion'}, inplace = True)

In [36]:
postulaciones.sample(1)

,idaviso,idpostulante
5508845,1112367832,0zPz5N8


In [37]:
#armo un nuevo df que tenga idaviso | #postulantes
cant_postulantes_por_aviso = postulaciones.groupby(['idaviso',]).size().to_frame('cant_postulaciones').reset_index()
#lo agrupo para tener el top de avisos
cant_postulantes_por_aviso.sort_values('cant_postulaciones', inplace = True, ascending = False)
cant_postulantes_por_aviso

,idaviso,cant_postulaciones
10405,1112334791,24420
1084,1112094756,22424
931,1112033906,20570
10402,1112334788,16687
1640,1112196813,13619
6977,1112280937,12124
447,1111753681,11102
1837,1112204682,10098
11224,1112345900,9784
766,1111970596,9746


In [38]:
cant_postulantes_por_aviso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20231 entries, 10405 to 1874
Data columns (total 2 columns):
idaviso               20231 non-null int64
cant_postulaciones    20231 non-null int64
dtypes: int64(2)
memory usage: 474.2 KB


In [39]:
def to_dic(clave, valor, df):
    dic = {}
    for index, row in df.iterrows():
        key = row[clave]
        value = row[valor]
        if (key not in dic):
            dic[key]=[value]
        else:
            if (value not in dic[key]):
                dic[key].append(value)
    return dic

In [43]:
#t0=time()

#dic_aviso_listidpost = to_dic('idaviso','idpostulante',postulaciones)

#tf = time() - t0
#print("se corrio en:")
#print(tf)

In [44]:
#len(dic_aviso_listidpost)

In [40]:
#agrego una columna de se postulo
postulaciones['sepostulo_x'] = 1

postulaciones.sample(10)

,idaviso,idpostulante,sepostulo_x
5117106,1112376814,owz6jKa,1
1304443,1112297717,pzelVKL,1
333817,1111706972,mzVdXl3,1
1941058,1112346741,8MPWRKL,1
4461950,1112422965,2zPxwmv,1
2803079,1112281810,BmDVGzE,1
5784248,1112422649,0zBRw6Y,1
374685,1112351502,EzEzpRo,1
3247723,1112286715,QNrR1Gl,1
542543,1112282505,2zPQORa,1


In [44]:
#df_aviso_listidpost = pd.DataFrame(list(dic_aviso_listidpost.items()), columns=['idaviso','lista_posulantes'])
#df_aviso_listidpost.head(10)


### DF 6: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [41]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
11146,1112317240,1,Vendedores en Gba Norte,<p>Nos encontramos en la búsqueda de vendedore...,Gran Buenos Aires,NaN,NaN,Part-time,Junior,Ventas,EAYA Consulting
4511,1112227290,1,Ganá sin manejar - Afiliador de conductores CA...,"<p style=""""><span style="""">GANÁ SIN MANEJAR ! ...",Gran Buenos Aires,NaN,NaN,Teletrabajo,Senior / Semi-Senior,Ventas,UBER
9532,1112311784,1,Analista de COMPRAS SemiSenior ( zona VICENTE ...,<p>En KaizenRH buscamos URGENTE <strong>Analis...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Compras,Kaizen Recursos Humanos
3660,1112093215,1,Coordinador para Obras Eléctricas. (Ingeniero ...,"<p align=""left"" style=""""><span style="""">Para i...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,HuCap
4153,1112291721,1,PROGRAMADOR DE CENTRO MECANIZADO,<p>COMPLEMENTOS selecciona PROGRAMADOR DE CENT...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,Complement Group (holding)


In [42]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25563 entries, 0 to 25562
Data columns (total 11 columns):
idaviso                 25563 non-null int64
idpais                  25563 non-null int64
titulo                  25563 non-null object
descripcion             25563 non-null object
nombre_zona             25563 non-null object
ciudad                  137 non-null object
mapacalle               1854 non-null object
tipo_de_trabajo         25563 non-null object
nivel_laboral           25563 non-null object
nombre_area             25563 non-null object
denominacion_empresa    25556 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [43]:
#elimino las columnas nombre_empresa y descripcion, ya que no vamos a usar esa info
#podriamos eliminar titulo tambien, esperar a si hacemos TF-IDF
avisos_detalles.drop(columns={'descripcion','denominacion_empresa'}, inplace=True)

In [44]:
avisos_detalles.sample(5)

,idaviso,idpais,titulo,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area
3753,1112355555,1,JAVA/CLOUD DEVELOPER - Full Stack - Senior,Capital Federal,NaN,retiro,Full-time,Senior / Semi-Senior,Programación
10392,1112445745,1,Comprador Senior,Gran Buenos Aires,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Compras
19420,1111927667,1,TÉCNICO INSTALADORES para RESISTENCIA / CORRIE...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Soporte Técnico
14554,1112460712,1,Gerente Comercial,Gran Buenos Aires,NaN,NaN,Full-time,Gerencia / Alta Gerencia / Dirección,Ventas
24454,1112407510,1,Responsable de Recursos Humanos,Gran Buenos Aires,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,Recursos Humanos


Eliminamos los avisos duplicados

In [45]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad' y 'mapacalle'.

In [46]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

1    24950
Name: idpais, dtype: int64

In [47]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

True     24818
False      132
Name: ciudad, dtype: int64

In [48]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

True     23152
False     1798
Name: mapacalle, dtype: int64

In [49]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [50]:
avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              22972
Capital Federal                 1914
Buenos Aires (fuera de GBA)       42
La Plata                           4
GBA Oeste                          3
Mendoza                            3
Ciudad de Mendoza                  3
Rosario                            2
Catamarca                          1
Santa Fe                           1
San Juan                           1
Cordoba                            1
Tucuman                            1
Neuquen                            1
Santa Cruz                         1
Name: nombre_zona, dtype: int64

In [51]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

Gran Buenos Aires = 3
GBA Oeste = 3
La Plata = 3
Capital Federal = 2
Buenos Aires (fuera de GBA) = 2
Otros = 1

In [52]:
avisos_detalles['tipo_de_trabajo'].value_counts()

Full-time          22523
Part-time           1725
Teletrabajo          248
Por Horas            124
Pasantia             119
Temporario            89
Por Contrato          87
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [53]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [54]:
avisos_detalles['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16977
Junior                                   4149
Otro                                     1975
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [55]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

In [56]:
avisos_detalles.sample(5)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
16143,1112270088,Diseñador Gráfico,3,9,3,Diseño Gráfico
3913,1112421812,ADMINISTRATIVO SR PARA IMPORTANTE GRUPO FARMAC...,2,10,3,Administración
16726,1112402769,Ejecutivo de Campo Jr.,3,10,2,Comercial
17139,1112338408,Pasante Mejora Continua,3,9,2,Pasantía / Trainee
23968,1111476991,Playeros/as para importante estación de servicio,3,10,3,Atención al Cliente


In [57]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 6 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
dtypes: int64(4), object(2)
memory usage: 1.3+ MB


Liberamos memoria para mejorar la performance

In [58]:
lista=0
fecha=0
postulaciones_2=0
avisos_detalles_2=0
postulantes_educ=0
postulantes_gen_nac=0

Creamos un dataframe que contenga todas las combinaciones de idpostulante e idaviso posibles. Inicialmente tendrán un 0 en la columna 'sepostulo'

In [ ]:
#OJO NO CORRER

In [64]:
t0 = time()

ids_postulantes = np.array(postulantes['idpostulante'])
ids_avisos = np.array(avisos_detalles['idaviso'])

tf = time() - t0
print("Tiempo de ejecución:", tf)


Tiempo de ejecución: 0.007719993591308594


In [ ]:
t0 = time()

dic_avisos_postulantes = {}

for idaviso in ids_avisos:
    for idpostulante in ids_postulantes:
        if (idaviso not in dic_avisos_postulantes):
            dic_avisos_postulantes[idaviso] = [idpostulante]
        else:
            dic_avisos_postulantes[idaviso].append(idpostulante)

tf = time() - t0
print("Tiempo de ejecución:", tf)


In [ ]:
t0 = time()

lista_avisos_postulantes = []

for idaviso in dic_avisos_postulantes.keys():
    for idpostulante in dic_avisos_postulantes[idaviso]:
        tupla = (idaviso, idpostulante)
        lista_avisos_postulantes.append(tupla)
        
tf = time() - t0
print("Tiempo de ejecución:", tf)


In [ ]:
df_avisos_postulantes_all = pd.DataFrame(lista_avisos_postulantes, columns=['idaviso','idpostulante'])
df_avisos_postulantes_all['sepostulo_y'] = 0

df_avisos_postulantes_all.head(10)

In [ ]:
df_avisos_postulantes_all.info()

In [ ]:
modelo = pd.merge(df_avisos_postulantes_all, postulaciones, on=['idaviso','idpostulante'],\
                  how='outer')

modelo.sample(5)

In [ ]:
modelo.info()

In [ ]:
modelo['sepostulo'] = modelo['sepostulo_x'] + modelo['sepostulo_y']
modelo.drop(columns={'sepostulo_x','sepostulo_y'}, inplace=True)

modelo.sample(5)

In [ ]:
modelo.info()

In [ ]:
modelo['sepostulo'].value_counts()

In [ ]:
#WARNING; CORRI HASTA ACA

In [ ]:
modelo.rename(columns={'estudios':'nivel_estudios'},inplace=True)
modelo.info()

Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

In [ ]:
modelo.info()

Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [ ]:
modelo = modelo[['idaviso','idpostulante','rango_edad','sexo',\
                 'nivel_estudios','esta_estudiando','titulo','tipo_de_trabajo',\
                 'nivel_laboral','nombre_zona','nombre_area','sepostulo']]

modelo.sample(5)

In [ ]:
modelo.info()

Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el dataframe.

In [ ]:
modelo["sepostulo"].value_counts()

Liberamos memoria para mejorar la performance

In [ ]:
postulantes=0
avisos_detalles=0


Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones

In [ ]:
no = modelo["sepostulo"] == 0
si = modelo["sepostulo"] == 1

modelo_no = modelo[no]
modelo_si = modelo[si]

In [ ]:
n_samples = 500000

modelo_no = modelo_no.sample(n_samples)
modelo_si = modelo_si.sample(n_samples)

modelo_final = pd.merge(modelo_no, modelo_si, how='outer')

modelo_final.sample(5)

Transformamos las variables categóricas de entero, objeto o flotante a categóricas propiamente dichas

In [ ]:
modelo_final['rango_edad'] = modelo_final['rango_edad'].astype('category')
modelo_final['sexo'] = modelo_final['sexo'].astype('category') 
modelo_final['nivel_estudios'] = modelo_final['nivel_estudios'].astype('category')
modelo_final['esta_estudiando'] = modelo_final['esta_estudiando'].astype('category')
modelo_final['tipo_de_trabajo'] = modelo_final['tipo_de_trabajo'].astype('category')
modelo_final['nivel_laboral'] = modelo_final['nivel_laboral'].astype('category')
modelo_final['nombre_zona'] = modelo_final['nombre_zona'].astype('category')
modelo_final['sepostulo'] = modelo_final['sepostulo'].astype('category')

In [ ]:
modelo_final.info()

Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [ ]:
modelo_final.to_csv("modelo_final.csv", encoding = "utf-8", index = False)